In [ ]:
import json
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
import matplotlib.pyplot as plt
from glob import glob
import cv2

In [ ]:
train_waypoint = pd.read_csv('../input/indoor-location-train-waypoints/train_waypoints.csv')
hand_waypoint = pd.read_csv('../input/waypoint_by_hand.csv')

In [ ]:
train_waypoint

In [ ]:
hand_waypoint

In [ ]:
# metaデータが存在するディレクトリ
DIR_META = '../input/indoor-location-navigation/metadata/'

# 画像の解像度(100とか200がいい感じかな。debugのときは一時的に10とかにすることもあります)
dpi = 100

In [ ]:
def pil2cv(image):
    ''' PIL型 -> OpenCV型 '''
    new_image = np.array(image, dtype=np.uint8)
    if new_image.ndim == 2:  # モノクロ
        pass
    elif new_image.shape[2] == 3:  # カラー
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR)
    elif new_image.shape[2] == 4:  # 透過
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGBA2BGRA)
    return new_image

In [ ]:
import io

def get_metadata(site, floor):
    # geojsonファイルを使ってsiteの一番外枠を取得したい
    path_json = f'{DIR_META}{site}/{floor}/geojson_map.json'
    with open(path_json) as json_file:
        map = json.load(json_file)

    # 座標
    coordinates = map['features'][0]['geometry']['coordinates']
    x_list = []
    y_list = []
    for i in range(len(coordinates)):
        ndim = np.ndim(coordinates[i])
        if ndim ==2:
            x = np.array(coordinates[i])[:,0]
            y = np.array(coordinates[i])[:,1]
        elif ndim==3:
            x = np.array(coordinates[i])[0,:,0]
            y = np.array(coordinates[i])[0,:,1]
        else:
            print('dim error')
        x_list.append(x)
        y_list.append(y)
    x = np.concatenate(x_list)
    y = np.concatenate(y_list)
    xmax,xmin,ymax,ymin = max(x),min(x),max(y),min(y)
    x = x - x.min()
    y = y - y.min()
    
    # 実際のfloor画像に合体させる
    img_path = f'{DIR_META}{site}/{floor}/floor_image.png'
    img = cv2.imread(img_path, -1)  # alphaも読み取るため-1を与える (4次元)
    index = np.where(img[:, :, 3] == 0)  # alpha=0の箇所を抽出
    img[index] = [255, 255, 255, 255]  # 抽出した箇所を白塗り
    img = img[:,:,[0,1,2]]  # 4->3次元にする
    height = img.shape[1]
    width = img.shape[0]

    # スケールを揃える
    scale_x = height/(xmax - xmin)
    scale_y = width/(ymax - ymin)
    scale_x, scale_y
    x = x*scale_x
    y = y*scale_y
    xy = np.stack([x, y], axis=1)

    return img, xy

In [ ]:
def create_auto_waypoint(site, floor):
    # 2つの画像をmergeして後の処理をしやすくする
    img, _xy = get_metadata(site, floor)   # w, h逆にする
    img = pil2cv(img)
    img_HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img_H, img_S, img_V = cv2.split(img_HSV)

    # json
    path_json = f'{DIR_META}{site}/{floor}/floor_info.json'
    with open(path_json) as json_file:
        json_data = json.load(json_file)
    w = json_data["map_info"]["width"]
    h = json_data["map_info"]["height"]

    # detect tulips
    thr = 1   # thrより大きいものを255(white)にする
    _thre, img_flowers = cv2.threshold(img_H, thr, 255, cv2.THRESH_BINARY)

    h_scale = h/img.shape[0]  # 1pxあたりのスケール
    w_scale = w/img.shape[1]  # 1pxあたりのスケール

    xy_list = []
    # find tulips
    contours, hierarchy = cv2.findContours(img_flowers, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    for i in range(0, len(contours)):
        if len(contours[i]) > 0:
            # remove small objects
            if cv2.contourArea(contours[i]) < 100:  # 500
                continue
            for j in range(len(contours[i])):

                if j % 20 == 0:  # hyperpara
                    # 座標抽出
                    x = contours[i][j,0,0]
                    y = contours[i][j,0,1]
                    x = x * w_scale  
                    y = h - (y * h_scale)
                    xy_list.append(np.array([x,y]))
    xy = np.stack(xy_list)

    # スケールを合わせる
    _xy[:,0] = _xy[:,0] * w_scale  
    _xy[:,1] = _xy[:,1] * h_scale 
    return img, xy, _xy, w, h

In [ ]:
site = '5d2709d403f801723c32bd39'
floor = '1F'

img, xy, _xy, w, h = create_auto_waypoint(site, floor)
plt.figure(figsize=(10, 10))
plt.imshow(img, extent=(0, w, 0, h))  # スケールをextentで調整
plt.scatter(xy[:,0],xy[:,1], s=10, color='green')  # 内側
plt.scatter(_xy[:,0],_xy[:,1], s=10, color='green')  # 外側

In [ ]:
from tqdm.notebook import tqdm
df_list = []
map_floor = {'B2': -2, 'B1': -1, 'F1': 0, 'F2': 1, 'F3': 2, 'F4': 3, 'F5': 4, 'F6': 5, 'F7': 6, 'F8': 7, 'F9': 8,
                '2B': -2, '1B': -1, '1F': 0, '2F': 1, '3F': 2, '4F': 3, '5F': 4, '6F': 5, '7F': 6, '8F': 7, '9F': 8}
for site in tqdm(sorted(train_waypoint['site'].unique())):
    print(site)
    floor_list = sorted(glob(f'{DIR_META}{site}/*'))
    floor_list = [text.split('/')[-1] for text in floor_list]
    for floor in floor_list:
        _, xy1, xy2, _, _ = create_auto_waypoint(site, floor)
        # df = pd.DataFrame(np.concatenate([xy1, xy2]), columns=['x', 'y'])
        df = pd.DataFrame(xy1, columns=['x', 'y'])

        df['site'] = site
        df['floor'] = map_floor[floor]
        df_list.append(df)
auto_waypoint = pd.concat(df_list).reset_index(drop=True)
auto_waypoint


In [ ]:
def visualize_map(train_waypoint, hand_waypoint, auto_waypoint, show=True, dpi=100):
    map_floor = {'B2': -2, 'B1': -1, 'F1': 0, 'F2': 1, 'F3': 2, 'F4': 3, 'F5': 4, 'F6': 5, 'F7': 6, 'F8': 7, 'F9': 8,
                 '2B': -2, '1B': -1, '1F': 0, '2F': 1, '3F': 2, '4F': 3, '5F': 4, '6F': 5, '7F': 6, '8F': 7, '9F': 8}
 
    site_list = train_waypoint['site'].unique()

    # site loop
    for site in sorted(site_list):
        floor_list = sorted(glob(f'{DIR_META}{site}/*'))
        floor_list = [text.split('/')[-1] for text in floor_list]
        
        # 画像の比を決めるために一度floor_info.jsonを取得
        path_json = f'{DIR_META}{site}/{floor_list[0]}/floor_info.json'
        with open(path_json) as json_file:
            json_data = json.load(json_file)
        _width_meter = json_data["map_info"]["width"]
        _height_meter = json_data["map_info"]["height"]
        s = max(_width_meter, _height_meter)

        t_site_wp = train_waypoint.loc[train_waypoint['site']==site, :].reset_index(drop=True)
        h_site_wp = hand_waypoint.loc[hand_waypoint['site']==site, :].reset_index(drop=True)
        a_site_wp = auto_waypoint.loc[auto_waypoint['site']==site, :].reset_index(drop=True)
        # figとaxesの用意
        n_floor = len(floor_list)
        n_row = int(np.ceil(n_floor / 3))
        w = round(5*3*_width_meter/s)
        h = round(5*n_row*_height_meter/s)
        fig, axs = plt.subplots(n_row, 3, figsize=(w, h), dpi=dpi)
        fig.suptitle(f'{site} (train:{len(t_site_wp)} hand:{len(h_site_wp)})', fontsize=15)
        fig.patch.set_facecolor('white')
        axs = axs.ravel()

        # floor loop
        for i_floor, floor in enumerate(floor_list):
            # json
            path_json = f'{DIR_META}{site}/{floor}/floor_info.json'
            with open(path_json) as json_file:
                json_data = json.load(json_file)
            width_meter = json_data["map_info"]["width"]
            height_meter = json_data["map_info"]["height"]

            t_wp = t_site_wp.loc[t_site_wp['floor']==map_floor[floor], :].reset_index(drop=True)
            h_wp = h_site_wp.loc[h_site_wp['floor']==map_floor[floor], :].reset_index(drop=True)
            a_wp = a_site_wp.loc[a_site_wp['floor']==map_floor[floor], :].reset_index(drop=True)

            # show image
            path_img = f'{DIR_META}{site}/{floor}/floor_image.png'
            im = Image.open(path_img)
            axs[i_floor].imshow(np.asarray(im), extent=(0, width_meter, 0, height_meter))
            axs[i_floor].set_title(f'{floor} (train:{len(t_wp)} hand:{len(h_wp)})', fontsize=10)

            # plot waypoint
            axs[i_floor].scatter(t_wp['x'], t_wp['y'], marker='o', color='red', s=1)
            axs[i_floor].scatter(h_wp['x'], h_wp['y'], marker='o', color='blue', s=1)
            axs[i_floor].scatter(a_wp['x'], a_wp['y'], marker='o', color='green', s=1)
        plt.tight_layout()
        # plt.savefig(f'./waypoint/{site}.png')
        plt.show()

In [ ]:
def visualize_onemap(train_waypoint, hand_waypoint, auto_waypoint, site, floor, dir_meta=DIR_META, show=True, dpi=100):
    map_floor = {'B2': -2, 'B1': -1, 'F1': 0, 'F2': 1, 'F3': 2, 'F4': 3, 'F5': 4, 'F6': 5, 'F7': 6, 'F8': 7, 'F9': 8,
                 '2B': -2, '1B': -1, '1F': 0, '2F': 1, '3F': 2, '4F': 3, '5F': 4, '6F': 5, '7F': 6, '8F': 7, '9F': 8}
    path_json = f'{dir_meta}{site}/{floor}/floor_info.json'
    with open(path_json) as json_file:
        json_data = json.load(json_file)
    width_meter = json_data["map_info"]["width"]
    height_meter = json_data["map_info"]["height"]

    # show image
    fig = plt.figure(figsize=(7,7))
    ax = fig.add_subplot(111)
    path_img = f'{dir_meta}{site}/{floor}/floor_image.png'
    im = Image.open(path_img)
    ax.imshow(np.asarray(im), extent=(0, width_meter, 0, height_meter))
    ax.set_title(floor, fontsize=10)

    # train&hand waypoint
    t_wp = train_waypoint.loc[(train_waypoint['site']==site)&(train_waypoint['floor']==map_floor[floor]), :].reset_index(drop=True)
    h_wp = hand_waypoint.loc[(hand_waypoint['site']==site)&(hand_waypoint['floor']==map_floor[floor]), :].reset_index(drop=True)
    ax.scatter(t_wp['x'], t_wp['y'], marker='o', color='red', s=5, label='train')
    ax.scatter(h_wp['x'], h_wp['y'], marker='o', color='blue', s=5, label='hand')

    # auto waypoint
    a_wp = auto_waypoint.loc[(auto_waypoint['site']==site)&(auto_waypoint['floor']==map_floor[floor]), :].reset_index(drop=True)
    ax.scatter(a_wp['x'], a_wp['y'], marker='o', color='green', s=10, label='auto')
    plt.legend()
    plt.tight_layout()
    # plt.savefig('auto_waypoint_sample.png')
    plt.show()

In [ ]:
site = '5d2709d403f801723c32bd39'
floor = '1F'
visualize_onemap(train_waypoint, hand_waypoint, auto_waypoint,site,floor)

In [ ]:
# auto_waypoint.to_csv('../input/auto_waypoint_v1.csv', index=False)

In [ ]:
# visualize submission
visualize_map(train_waypoint, hand_waypoint, auto_waypoint, dpi=dpi)

In [ ]:
auto_waypoint